In [299]:
#import dependencies
import pandas as pd
import pymongo
from bs4 import BeautifulSoup
from splinter import Browser

In [300]:
#connect to mongodb port
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [301]:
executable_path = {'executable_path': './chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [302]:
# Visit the NASA Mars News Site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [303]:
# Scrape page into Soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [304]:
# Find the latest news title and paragraph text

news_title = soup.find('div', class_='list_text').find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

Hear Audio From NASA's Perseverance As It Travels Through Deep Space
The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight.


### JPL Mars Space Images - Featured Image


In [320]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [322]:
# Click links to retrieve html of large feature image
browser.links.find_by_partial_text('FULL IMAGE').click()
browser.links.find_by_partial_text('more info').click()


In [323]:
# Use splinter to navigate the site and find the full size image url for the current Featured Mars Image
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

url = soup.find('figure', class_='lede').find('a')['href']
featured_image_url = 'https://www.jpl.nasa.gov' + url
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19347_hires.jpg'

### Mars Facts

In [309]:
url = 'https://space-facts.com/mars/'


In [310]:
# Use Pandas to scrape the table of mars facts
table = pd.read_html(url)[0]
table.columns=["Description", "Mars"]
mars_facts = table.set_index('Description')
mars_facts

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [311]:
#Use Pandas to convert the data to a HTML table string
mars_table = mars_facts.to_html().replace('\n', '')
mars_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

### Mars Hemispheres

In [329]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [330]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [331]:
# Scrape and List the names of the 4 hemispheres

#Source: "Scraping text in h3 and div tags using beautifulSoup, Python", edited Oct 25 '17 at 14:32, Anonta, 
#https://stackoverflow.com/questions/46933679/scraping-text-in-h3-and-div-tags-using-beautifulsoup-python
title = soup.find_all('h3')
hemispheres = [x.text.split(' Enhanced')[0] for x in title]
hemispheres

['Cerberus Hemisphere',
 'Schiaparelli Hemisphere',
 'Syrtis Major Hemisphere',
 'Valles Marineris Hemisphere']

In [332]:
hemisphere_image_urls = []

for hemisphere in hemispheres:
    hem_dict = {}
    # Click each of the links to the hemispheres in order to find the image url to the full resolution image
    browser.links.find_by_partial_text(hemisphere).click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    # Save image url string and the Hemisphere title containing the hemisphere name in a Python dictionary
    hem_dict['title'] = soup.find('h2', class_='title').text.replace("Enhanced","").strip()
    hem_dict['img_url'] = soup.find('div', class_='downloads').find('a')['href']
    hemisphere_image_urls.append(hem_dict)
    
    browser.back()
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [333]:
browser.quit()